In [4]:
%matplotlib widget
import scipy.io as io
import numpy as np
import matplotlib.pyplot as plt



#数据读取与质量预处理
data = io.loadmat('../../Matlab_files/top3d_Model/top3d_Model_1/iter-100/iters-100-006/topModel_1_100_006.mat')
data = data['data']
data = np.array(data)
print('数据初始维度：',data.shape)
data = data.transpose(1, 2, 0)
print('数据调整后维度：',data.shape)#(x,z,y)
for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        for k in range(data.shape[2]):
            if data[i][j][k] > 0.5:
                data[i][j][k] = 1
            else:
                data[i][j][k] = 0



#数据尺寸扩展为标准尺寸
nelx, nely, nelz = 120, 120, 120
if (nelx - data.shape[0])%2 !=0:
    data = np.pad(data, ((1, 0), (0, 0), (0 ,0)))
if (nelz - data.shape[1])%2 !=0:
    data = np.pad(data, ((0, 0), (1, 0), (0 ,0)))
if (nely - data.shape[2])%2 !=0:
    data = np.pad(data, ((0, 0), (0, 0), (1 ,0)))
data_new = np.pad(data, (((nelx - data.shape[0])//2, (nelx - data.shape[0])//2),
                    ((nely - data.shape[1])//2, (nely - data.shape[1])//2),
                    ((nelz - data.shape[2])//2, (nelx - data.shape[2])//2)))
print('数据标准化后维度：',data_new.shape)



#尺寸标准化后模型可视化
#测试保存
# io.savemat('C:/Users/11573/Desktop/data/standardinizetest.mat',{'standarddata':data_new})
#正式保存
io.savemat('../../Matlab_files/top3d_Model/top3d_Model_1/iter-100/iters-100-006/standardtopModel_1_100_006.mat',{'standarddata':data_new})

#查找模型存在的单共享边等断裂结构
defect_data = np.zeros((120, 120, 120))
face_voxel = []
search_list = np.zeros((1, 6, 3))
n = 0
t = 0
l = 0
defect_model = np.zeros((nely, nelx, nelz))
defect_index = np.array([[0, 0, 0]])
for i in range(1,(data_new.shape[0]-1)):
    for j in range(1,(data_new.shape[1]-1)):
        for k in range(1,(data_new.shape[2]-1)):
            if data_new[i][j][k] !=0:
                face_voxel.append(data_new[i-1][j][k])
                # print('1:',face_voxel)
                face_voxel.append(data_new[i+1][j][k])
                # print('2:',face_voxel)
                face_voxel.append(data_new[i][j-1][k])
                # print('3:',face_voxel)
                face_voxel.append(data_new[i][j+1][k])
                # print('4:',face_voxel)
                face_voxel.append(data_new[i][j][k-1])
                # print('5:',face_voxel)
                face_voxel.append(data_new[i][j][k+1])
                # print('6:',face_voxel)
                sum_value = sum(face_voxel)
                # print('face_voxel:',face_voxel)
                
                if sum_value == 0 or sum_value == 1:
                    x, y, z = i, j, k
                    defect_array = np.array([[x, y ,z]])
                    # print('i,j,k:',i, j, k)
                    defect_index = np.append(defect_index, defect_array, axis = 0)
                    
                    # defect_index = np.array(defect_index)
                    # print(defect_index.shape)
                    defect_data[i-2:i+2,j-2:j+2,k-2:k+2] = data_new[i-2:i+2,j-2:j+2,k-2:k+2]
                    face_voxel_index = np.array([[[x-1, z, y],
                                                 [x+1, z, y],
                                                 [x, z-1, y],
                                                 [x, z+1, y],
                                                 [x, z, y-1],
                                                 [x, z, y+1]]])
                    search_list = np.append(search_list, face_voxel_index, axis=0)
                    l += 1
                   
                face_voxel = []
                t += 1    
            n += 1

#制作在matlab中显示的缺陷中心点
for t1 in range(1, len(defect_index)):
    exam_index = defect_index[t1]
    x1, y1, z1 = exam_index[0], exam_index[1], exam_index[2]
    defect_model[x1][y1][z1] = 1
defect_data = defect_data + defect_model
# Top3dSTL_v3('C:\Users\11573\Desktop\VSCode_repos\Generate_dataset\Matlab_files\top3d_Model_1\repairtopModel_1.STL',repairdata,'Format','binary','Mode','iso')
#修复模型

# data_new[49][57][58] = 1
# data_new[49][62][58] = 1
# data_new[55][58][62] = 1
# data_new[55][61][62] = 1
# data_new[56][56][60] = 1
# data_new[56][63][60] = 1
# data_new[58][54][58] = 1
# data_new[58][65][58] = 1
# data_new[62][55][56] = 0
# data_new[62][64][56] = 0
# data_new[64][58][56] = 1
# data_new[64][61][56] = 1
# data_new[63][53][65] = 1
# data_new[63][66][65] = 1
# data_new[71][56][57] = 1
# data_new[71][63][57] = 1
# data_new[58][55][60] = 1
# data_new[58][64][60] = 1
# data_new[59][55][59] = 1
# data_new[59][64][59] = 1
# data_new[59][54][59] = 1
# data_new[59][65][59] = 1
# data_new[59][53][58] = 1
# data_new[59][66][58] = 1
# data_new[59][55][68] = 1
# data_new[59][64][68] = 1
# data_new[61][55][66] = 1
# data_new[61][64][66] = 1
# data_new[63][54][66] = 1

# data_new[62][54][58] = 1
# data_new[62][65][58] = 1
# data_new[62][53][58] = 1
# data_new[62][66][58] = 1
# data_new[62][54][56] = 1
# data_new[62][65][56] = 1
# data_new[62][55][55] = 1
# data_new[62][64][55] = 1

# data_new[65][52][62] = 1
# data_new[65][67][63] = 1
# data_new[67][53][63] = 1

# data_new[69][53][64] = 1
# data_new[69][65][64] = 1
# data_new[68][53][61] = 1


# data_new[70][56][57] = 1
# data_new[69][63][57] = 1

# data_new[71][58][57] = 1
# data_new[71][61][57] = 1
# data_new[71][57][67] = 1

# data_new[73][57][67] = 1
# data_new[73][62][67] = 1

# data_new[76][60][66] = 1

# data_new[81][60][63] = 1
# data_new[83][60][63] = 1
print(defect_index)
print(len(defect_index))
# print('遍历了：',n,'次')
# print('共有',t,'个非零元素')
# print('共有',l,'个缺陷点')
# print('search_list:',search_list)
# print('search_list.shape:',search_list.shape)


# # #针对模型缺陷部分进行插补
# for o in range(1,len(search_list)):970
#     repair_index = search_list[o]
#     for w in range(6):
#         point_index = repair_index[w]
#         index_x = int(point_index[0])
#         index_z = int(point_index[1])
#         index_y = int(point_index[2])
#         data_new[index_x, index_z, index_y] = 1

#测试保存
# io.savemat('C:/Users/11573/Desktop/data/defecttopModel.mat',{'defectdata':defect_data})

# #正式保存
# io.savemat('../../Matlab_files/top3d_Model_7/iters-180/iters-180-004/defecttopModel_7_180_004.mat',{'defectdata':defect_data})
# io.savemat('../../Matlab_files/top3d_Model_7/iters-180/iters-180-004/repairtopModel_7_180_004.mat',{'repairdata':data_new})


数据初始维度： (20, 60, 16)
数据调整后维度： (60, 16, 20)
数据标准化后维度： (120, 120, 120)
[[ 0  0  0]
 [42 57 62]
 [42 62 62]
 [68 52 56]
 [68 53 57]
 [68 66 57]
 [68 67 56]
 [69 53 58]
 [69 54 59]
 [69 54 61]
 [69 65 59]
 [69 65 61]
 [69 66 58]
 [70 55 61]
 [70 55 63]
 [70 56 64]
 [70 63 64]
 [70 64 61]
 [70 64 63]
 [71 56 65]
 [71 57 66]
 [71 62 66]
 [71 63 65]
 [72 57 67]
 [72 58 68]
 [72 61 68]
 [72 62 67]
 [89 58 69]
 [89 61 69]]
29
